[View in Colaboratory](https://colab.research.google.com/github/Philipid3s/MusicGeneratorWithTensorFlow/blob/master/NES_Model_TensorFlow.ipynb)

# Chiptune music auto composer with deep learning (TensorFlow RNN Model)

### 1- Import some NES MIDI files to feed our dataset
MIDI files imported from [nesmdb](https://github.com/chrisdonahue/nesmdb)

In [2]:
!pip install nesmdb==0.1.8
!pip install pretty_midi
!wget http://deepyeti.ucsd.edu/cdonahue/nesmdb/nesmdb_midi.tar.gz
!tar xvfz nesmdb_midi.tar.gz

    100% |████████████████████████████████| 30.8MB 686kB/s 
    100% |████████████████████████████████| 2.0MB 9.7MB/s 
  Running setup.py bdist_wheel for nesmdb ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/91/26/52/2ba385cfc747c8c987f81a4a2344ef151a6d2f303e131cd275
Successfully built nesmdb
  Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
    100% |████████████████████████████████| 5.6MB 829kB/s 
    100% |████████████████████████████████| 71kB 14.1MB/s 
  Running setup.py bdist_wheel for pretty-midi ... - \ done
  Stored in directory: /root/

In [5]:
# Find MIDIs
import glob
midi_list = glob.glob('nesmdb_midi/train/*')
print 'full dataset size: ' + str(len(midi_list))

full dataset size: 4502


### 2 - Build the training dataset
NES music is composed of 4 voices P1, P2, TR and NO

We will build a predictive model only based on the first voice P1

In [103]:
# Analyze the MIDI data for instrument 1 (P1)

import pretty_midi

selected_files = []

# we limit our selection to only 1000 Midi files
midi_list = midi_list[:1000]

for midi_file in midi_list:
  try:
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    if (len(midi_data.instruments) > 0):
      if (len(midi_data.instruments[0].notes) > 100):
        # only midi files with more than 100 notes (relevant for the tests)
        print '{} --- Number of notes: {}'.format(midi_file, len(midi_data.instruments[0].notes))
        selected_files += [midi_data.instruments[0].notes]
  except ValueError:
      print "Oops!  Invalid Midi file format..."
  
print 'selected files: ' + str(len(selected_files))
  

nesmdb_midi/train/292_Shatterhand_11_12AreaG.mid --- Number of notes: 810
nesmdb_midi/train/228_MegaMan5_06_07WaveManStage.mid --- Number of notes: 150
nesmdb_midi/train/287_S_C_A_T__SpecialCyberneticAttackTeam_11_12Ending.mid --- Number of notes: 1097
nesmdb_midi/train/091_DragonWarriorIV_10_11ColosseumDressingRoom.mid --- Number of notes: 522
nesmdb_midi/train/276_RadiaSenki_ReimeiHen_24_25Ruins.mid --- Number of notes: 180
nesmdb_midi/train/078_Doraemon_10_11World22.mid --- Number of notes: 162
nesmdb_midi/train/080_DoubleDribble_01_02USAAnthem.mid --- Number of notes: 113
nesmdb_midi/train/280_RoadRunner_02_03Nutcracker.mid --- Number of notes: 140
nesmdb_midi/train/160_HiryunoKenIII_5NinnoRyuSenshi_11_12TalkwithDragonTusk.mid --- Number of notes: 632
nesmdb_midi/train/058_Contra_07_08FortressofIceStage5.mid --- Number of notes: 209
nesmdb_midi/train/050_ChaosWorld_21_22Ending.mid --- Number of notes: 1161
nesmdb_midi/train/053_ChoujinSentaiJetman_08_09Transformation.mid --- Number

In [63]:
import numpy as np

train_label = []
train_data = []

nb_prev_notes = 5
  
for mid_file in selected_files:
  list_notes = []
  
  for note in mid_file:
    list_notes += [ [ note.pitch, note.velocity ] ]

  idx = 1

  for note in list_notes:
    train = []
    if idx > nb_prev_notes:
      train_label += [ note ]
      train += list_notes[idx - 1 - 5]
      train += list_notes[idx - 1 - 4]
      train += list_notes[idx - 1 - 3]
      train += list_notes[idx - 1 - 2]
      train += list_notes[idx - 1 - 1]
      train_data += [ train ]
    idx+=1


train_data = np.array(train_data)
train_label = np.array(train_label)

print "total entries in train_data: " + str(len(train_data))
print "total entries in train_label: " + str(len(train_label))
print "Training set: " + str(train_data.shape) 

print train_data[0]

total entries in train_data: 130347
total entries in train_label: 130347
Training set: (130347, 10)
[69  7 57  7 57  2 60  7 60  2]


### 3 - Normalize the training dataset

In [64]:
mean = train_data.mean(axis=0)
print 'Mean: ' + str(mean)
std = train_data.std(axis=0)
print 'Std: ' + str(std)

train_data = (train_data - mean) / std

print train_data[0]

Mean: [68.07463156  6.29083907 68.0857327   6.28615158 68.09988722  6.28417992
 68.11262246  6.28286804 68.12218923  6.28232334]
Std: [12.79441498  3.92192546 12.7933227   3.9199175  12.78998044  3.9199484
 12.79132556  3.92042196 12.79488175  3.92048568]
[ 0.07232597  0.18081958 -0.8665249   0.18210802 -0.86785803 -1.09291743
 -0.63422844  0.18292214 -0.63479987 -1.09229409]


### 4 - Build the model (TensorFlow/Keras RNN)

In [65]:
# build model
import tensorflow as tf
from tensorflow import keras

def build_model():
  model = keras.Sequential([
    keras.layers.Dense(128, activation=tf.nn.relu,
                       input_shape=(train_data.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(2)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer= optimizer,
                metrics=['mae'])
  return model

model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 128)               1408      
_________________________________________________________________
dense_19 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 130       
Total params: 9,794
Trainable params: 9,794
Non-trainable params: 0
_________________________________________________________________


### 5 - Train the model

In [66]:
EPOCHS = 10

# Store training stats
model.fit(train_data, train_label, epochs = EPOCHS)

Epoch 1/10
130347/130347 [==============================] - 4s 32us/step - loss: 82.5917 - mean_absolute_error: 4.2241
Epoch 2/10
130347/130347 [==============================] - 4s 30us/step - loss: 31.9928 - mean_absolute_error: 3.0776
Epoch 3/10
130347/130347 [==============================] - 4s 30us/step - loss: 31.1506 - mean_absolute_error: 3.0214
Epoch 4/10
130347/130347 [==============================] - 4s 30us/step - loss: 30.5778 - mean_absolute_error: 2.9863
Epoch 5/10
130347/130347 [==============================] - 4s 30us/step - loss: 30.1186 - mean_absolute_error: 2.9597
Epoch 6/10
130347/130347 [==============================] - 4s 30us/step - loss: 29.8045 - mean_absolute_error: 2.9419
Epoch 7/10
130347/130347 [==============================] - 4s 30us/step - loss: 29.4730 - mean_absolute_error: 2.9235
Epoch 8/10
130347/130347 [==============================] - 4s 30us/step - loss: 29.2437 - mean_absolute_error: 2.9094
Epoch 9/10
130347/130347 [======================

### 6 - Build test dataset

In [0]:
auto_chiptune = [ [46, 15], [39, 15], [60, 10], [53, 15], [46, 15] ]

test_data = np.array([[ 46, 15, 39, 15, 60, 10, 53, 15, 46, 15]]) 

# normalize test dataset
test_data = (test_data - mean) / std

### 7 - Make predictions
Each predicted note will be based on the 5 previous notes


In [100]:
for i in range(1, 15):
  predict = model.predict(test_data).flatten()
  predict = predict.astype(int).tolist()
  print 'predicted note: ' + str(predict)
  auto_chiptune.append(predict)
  # normalize prediction
  normalized_predict = np.append([0,0,0,0,0,0,0,0], predict)
  normalized_predict = (normalized_predict - mean) / std
  test_data = np.array([np.append(test_data[0][2:], normalized_predict[8:])])

predicted note: [37, 14]
predicted note: [48, 12]
predicted note: [48, 13]
predicted note: [39, 13]
predicted note: [32, 12]
predicted note: [35, 11]
predicted note: [37, 11]
predicted note: [32, 12]
predicted note: [27, 11]
predicted note: [26, 10]
predicted note: [26, 10]
predicted note: [25, 10]
predicted note: [22, 10]
predicted note: [20, 10]


### 8 - Create MIDI track and play it !

In [101]:
import pretty_midi
import tempfile

bpm = 300
nbeats = len(auto_chiptune)

bps = (bpm / 60.)
nsecs = nbeats / bps
lenbeats = 0.5

# Create MIDI instruments
p1 = pretty_midi.Instrument(0, name='p1')

beat = 0

for note in auto_chiptune:
  print 'Note[{}] pitch:{} velocity:{}'.format(beat+1, note[0], note[1])
  start_t = float(beat) / bps
  end_t = (float(beat) + lenbeats) / bps
  # limit velocity : 15
  if (note[1] > 15):
    velocity = 15
  else:
    velocity = int(note[1])
  # limit picth 127
  if (int(note[0]) > 127):
    pitch = 127
  else:
    pitch = int(note[0])
  p1.notes.append(pretty_midi.Note(velocity=velocity, pitch=pitch, start=start_t, end=end_t))
  beat+=1

# Create MIDI file
midi = pretty_midi.PrettyMIDI(initial_tempo=120, resolution=22050)
midi.instruments.extend([p1])
midi.time_signature_changes.append(pretty_midi.TimeSignature(1, 1, nsecs))

# Write MIDI file and read binary
mf = tempfile.NamedTemporaryFile('rb')

midi.write(mf.name)
midi = mf.read()
mf.close()

# Convert to WAV and display
from nesmdb.convert import midi_to_wav
wav = midi_to_wav(midi, midi_to_wav_rate=None)
from IPython.display import display, Audio
display(Audio(wav[:44100 * 10], rate=44100))

Note[1] pitch:46 velocity:15
Note[2] pitch:39 velocity:15
Note[3] pitch:60 velocity:10
Note[4] pitch:53 velocity:15
Note[5] pitch:46 velocity:15
Note[6] pitch:37 velocity:14
Note[7] pitch:48 velocity:12
Note[8] pitch:48 velocity:13
Note[9] pitch:39 velocity:13
Note[10] pitch:32 velocity:12
Note[11] pitch:35 velocity:11
Note[12] pitch:37 velocity:11
Note[13] pitch:32 velocity:12
Note[14] pitch:27 velocity:11
Note[15] pitch:26 velocity:10
Note[16] pitch:26 velocity:10
Note[17] pitch:25 velocity:10
Note[18] pitch:22 velocity:10
Note[19] pitch:20 velocity:10
